In [1]:
!python -V

Python 3.12.3


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error
import pickle

In [4]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/oliver/Desktop/Workspace/Projects/mlops-zoomcamp-test/02-experiment_tracking/mlruns/1', creation_time=1716530987985, experiment_id='1', last_update_time=1716530987985, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe("./homework/data/green_tripdata_2023-01.parquet")
df_valid = read_dataframe("./homework/data/green_tripdata_2023-02.parquet")

In [7]:
len(df_train), len(df_valid)

(65946, 62574)

In [8]:
categorical = ['PULocationID', 'DOLocationID']
numerical = []

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_valid[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_valid[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

7.356159723684207

In [11]:
with open("models/lin_reg.bin", 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [12]:
with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

In [14]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "oliver")
    mlflow.log_param("train-data-path", "data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "data/yellow_tripdata_2023-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha=alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact("models/lin_reg.bin", artifact_path="models_pickle")
    mlflow.log_artifact("models/preprocessor.b", artifact_path="models_pickle")

### XGBoost

In [15]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [17]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50,
        )
        
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        
        return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # exp(-3) to exp(0) => 0.05 to 1
    'reg_apha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [18]:
best_params = {
    'learning_rate': 0.09585355369315604,
    'max_depth': 30,
    'min_child_weight': 1.060597050922164,
    'objective': 'reg:linear',
    'reg_alpha': 0.018060244040060163,
    'reg_lambda': 0.011658731377413597,
    'seed': 42
}

mlflow.xgboost.autolog()
booster = xgb.train(
    best_params,
    train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50
)

2024/05/24 17:23:51 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'f08b0611128f40a7862ee3888d9518a4', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/home/oliver/miniconda3/envs/mlops/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [17:23:51] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.00978
[1]	validation-rmse:8.74946
[2]	validation-rmse:8.54150
[3]	validation-rmse:8.36812
[4]	validation-rmse:8.18329
[5]	validation-rmse:8.05539
[6]	validation-rmse:7.90600
[7]	validation-rmse:7.81183
[8]	validation-rmse:7.70610
[9]	validation-rmse:7.63388
[10]	validation-rmse:7.55500
[11]	validation-rmse:7.49649
[12]	validation-rmse:7.44363
[13]	validation-rmse:7.40122
[14]	validation-rmse:7.35554
[15]	validation-rmse:7.29812
[16]	validation-rmse:7.26498
[17]	validation-rmse:7.22929
[18]	validation-rmse:7.20283
[19]	validation-rmse:7.16681
[20]	validation-rmse:7.13967
[21]	validation-rmse:7.11649
[22]	validation-rmse:7.06753
[23]	validation-rmse:7.04657
[24]	validation-rmse:7.02571
[25]	validation-rmse:7.00914
[26]	validation-rmse:6.98574
[27]	validation-rmse:6.96708
[28]	validation-rmse:6.93577
[29]	validation-rmse:6.91341
[30]	validation-rmse:6.87381
[31]	validation-rmse:6.86005
[32]	validation-rmse:6.84608
[33]	validation-rmse:6.82736
[34]	validation-rmse:6.8

2024/05/24 17:24:15 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/24 17:24:15 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/oliver/miniconda3/envs/mlops/lib/python3.12/site-packages/xgboost/core.py:160: UserWarning: [17:24:15] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


In [ ]:
with mlflow.start_run():

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)
    booster = xgb.train(
        best_params,
        train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [19]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./homework/data/green_tripdata_2023-01.csv")
        mlflow.log_param("valid-data-path", "./homework/data/green_tripdata_2023-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)   

2024/05/24 17:24:25 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/24 17:24:25 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/24 17:25:04 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/24 17:25:07 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/24 17:25:47 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
